  # IRBB Model Gap Analysis Repricing

The desired purpose of this code is to be able to record instruments and calculate the Gap Analysis Repricing, using a cash flow generator, visulazing the gap analysis.
~~
The code will be partioned in the following way:
1. DataFrame where instruments (Bonds) will be recorded (inputed) and given an identification symbol.
2. A cashflow generating script which will be used to calculate Gap Analysis.
3. A visulazing (Line Graph) script where the timestamped intruments will be delegated into buckets, the script will visualize the conclusions made in part 2.

## Part 1
DataFrame where instruments (Bonds) will be recorded (inputed) and given an identification symbol.

The following variables will be used as input, to calculate Cash Flows:
1. ID
2. Type
3. Name
4. Face Value
5. Coupon Rate
6. Period
7. Currency
8. Start Date
9. End Date
10. Maturity
11. Compound
12. Coupon Start

### Setting Variables


Let's define the above specified variables in a random example using 3 bonds. For future reference let's define a User Defined Function that is available to append instruments when user gives parameters for variables.

The expiration date is either the last day or only day on which the instrument can be exercised. In contrast, the maturity date is the day on which the underlying transaction settles when an instrument is applied.

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

variables = ['ID','Type','Name','Face_value','Coupon Rate','Period','Currency','Start_Date','End_Date','Maturity','Compound','Coupon Start']
print(variables)

ID = [1,2,3]
Type = ['Bond','Bond','Bond']
Name = ['Bond A', 'Bond B', 'Bond C']
#Dollar Amoount
Face_Value = [1e4,1e6,1e5]
#Percentage
Coupon_Rate = [3.00,3.25,2.88]
# In Months
Coupon_Period = [12,12,12]
Currency = ['USD','EUR','EUR']
Start_date = ['2.1.2023','2.1.2023','2.1.2023']
Maturity = ['2.1.2025','2.1.2028','2.1.2033']
Compound = ['M','M','M']
Coupon_Start = ['2.1.2023','15.5.2023','2.1.2023']
Expiration_Date = ['1.1.2025','1.1.2028','1.1.2033']

    

    

['ID', 'Type', 'Name', 'Face_value', 'Coupon Rate', 'Period', 'Currency', 'Start_Date', 'End_Date', 'Maturity', 'Compound', 'Coupon Start']


### Converting to DataFrame

The following code use the above supplied variables and creates a pandas DataFrame out of them.

In [4]:
variables_dict = {'ID':ID,
                  'Type':Type,
                  'Name':Name,
                  'Face Value':Face_Value,
                  'Coupon Rate':Coupon_Rate,
                  'Coupon Period':Coupon_Period,
                  'Currency':Currency,
                  'Start Date':Start_date,
                  'Maturity':Maturity,
                  'Compound':Compound,
                 'Coupon Start':Coupon_Start,
                 'Expiration Date':Expiration_Date}
print(variables_dict)




{'ID': [1, 2, 3], 'Type': ['Bond', 'Bond', 'Bond'], 'Name': ['Bond A', 'Bond B', 'Bond C'], 'Face Value': [10000.0, 1000000.0, 100000.0], 'Coupon Rate': [3.0, 3.25, 2.88], 'Coupon Period': [12, 12, 12], 'Currency': ['USD', 'EUR', 'EUR'], 'Start Date': ['2.1.2023', '2.1.2023', '2.1.2023'], 'Maturity': ['2.1.2025', '2.1.2028', '2.1.2033'], 'Compound': ['M', 'M', 'M'], 'Coupon Start': ['2.1.2023', '15.5.2023', '2.1.2023'], 'Expiration Date': ['1.1.2025', '1.1.2028', '1.1.2033']}


In [5]:


variables_df = pd.DataFrame.from_dict(variables_dict)
variables_df.set_index('ID')

,Type,Name,Face Value,Coupon Rate,Coupon Period,Currency,Start Date,Maturity,Compound,Coupon Start,Expiration Date
ID,,,,,,,,,,,
1,Bond,Bond A,10000.0,3.00,12,USD,2.1.2023,2.1.2025,M,2.1.2023,1.1.2025
2,Bond,Bond B,1000000.0,3.25,12,EUR,2.1.2023,2.1.2028,M,15.5.2023,1.1.2028
3,Bond,Bond C,100000.0,2.88,12,EUR,2.1.2023,2.1.2033,M,2.1.2023,1.1.2033


In [6]:
variables_df['Start Date'] = pd.to_datetime(variables_df['Start Date'])
variables_df['Maturity'] = pd.to_datetime(variables_df['Maturity'])
variables_df['Coupon Start'] = pd.to_datetime(variables_df['Coupon Start'])
variables_df['Expiration Date'] = pd.to_datetime(variables_df['Expiration Date'])

print(variables_df)
print(type(variables_df['Start Date'][1]))
variables_df = pd.DataFrame.from_dict(variables_dict)
variables_df.set_index('ID')

   ID  Type    Name  Face Value  Coupon Rate  Coupon Period Currency  \
0   1  Bond  Bond A     10000.0         3.00             12      USD   
1   2  Bond  Bond B   1000000.0         3.25             12      EUR   
2   3  Bond  Bond C    100000.0         2.88             12      EUR   

  Start Date   Maturity Compound Coupon Start Expiration Date  
0 2023-02-01 2025-02-01        M   2023-02-01      2025-01-01  
1 2023-02-01 2028-02-01        M   2023-05-15      2028-01-01  
2 2023-02-01 2033-02-01        M   2023-02-01      2033-01-01  
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,Type,Name,Face Value,Coupon Rate,Coupon Period,Currency,Start Date,Maturity,Compound,Coupon Start,Expiration Date
ID,,,,,,,,,,,
1,Bond,Bond A,10000.0,3.00,12,USD,2.1.2023,2.1.2025,M,2.1.2023,1.1.2025
2,Bond,Bond B,1000000.0,3.25,12,EUR,2.1.2023,2.1.2028,M,15.5.2023,1.1.2028
3,Bond,Bond C,100000.0,2.88,12,EUR,2.1.2023,2.1.2033,M,2.1.2023,1.1.2033


Below is a user_defined function that appends new data (Bonds, Futures) to the variables_df dictionary and creates Pandas DataFrame. NEEDS MODIFICATION

In [7]:

#def append_data(ID,Type,Name,Face_value,Coupon_Rate,Period,Currency,Start_date):
#    new_dict = {'ID':ID,'Type':Type,'Name':Name,'Face Value':Face_value,'Coupon Rate':Coupon_Rate,'Period':Period,'Currency':Currency,'Start Date':Start_date}
#    df_new_dict = pd.DataFrame.from_dict(new_dict,['ID','Type','Name','Face Value','Coupon Rate',])
#    print(df_new_dict)
#    print(type(df_new_dict))
#    variables_df.append(df_new_dict)


#append_data(1,'a','b',100,3.3,12,'EUR','25-07-2022')


# Part 2

A cashflow generating script which will be used to calculate cashflows schedule.

In [8]:
print(variables_df)

   ID  Type    Name  Face Value  Coupon Rate  Coupon Period Currency  \
0   1  Bond  Bond A     10000.0         3.00             12      USD   
1   2  Bond  Bond B   1000000.0         3.25             12      EUR   
2   3  Bond  Bond C    100000.0         2.88             12      EUR   

  Start Date  Maturity Compound Coupon Start Expiration Date  
0   2.1.2023  2.1.2025        M     2.1.2023        1.1.2025  
1   2.1.2023  2.1.2028        M    15.5.2023        1.1.2028  
2   2.1.2023  2.1.2033        M     2.1.2023        1.1.2033  


The user-defined function called 'cashflow()' calculates the Expected Cash Flow using the following formula:
1. CASH_FLOW = COUPON RATE * FACE VALUE

In [9]:

def cashflow(ID):
    EX_cashflow = variables_df.iloc[ID,4]*variables_df.iloc[ID,3]
    EX_cashflow = EX_cashflow * 
    print(EX_cashflow)
cashflow(1)


SyntaxError: invalid syntax (2912025797.py, line 3)

The User defined function called 'disc_cashflow()' discounts the expected cash flow to the present. We use the following code to calculate discounted expected cash flow:
1. CASH FLOW / (1+r)^t

In [10]:
def disc_cashflow(ID):
    #flt_cashflow = float(cashflow(ID))
    discounted_cashflow =  variables_df.iloc[ID,4]*variables_df.iloc[ID,3] / (1 + variables_df.iloc[ID,4])**variables_df.iloc[ID,5]
    print(discounted_cashflow)
disc_cashflow(2)

0.024740688462267835


The user defined function called 'cash_flw_val' values the individual cash flows (Cash Flow Value)and a final face value payment, to value the bond as a whole. Using the following calculation:
1. CASH FLOW VALUE = CASH FLOW = (1+r)^1 + (1+r)^2 + (1+r)^3 + ....+ (1+r)^t

In [11]:
def cash_flw_Val(ID):
    Csh_Flw_Val = variables_df.iloc[ID,4]*variables_df.iloc[ID,3] / (1+variables_df.iloc[ID,4])**
    print(cash_flw_Val)

SyntaxError: invalid syntax (1443559415.py, line 2)

Converting string dates to DateTime & Calculating Cash Flow Period

In [12]:
EndDate
print(type(variables_df['Start Date'][1]))
s = variables_df['Coupon Start'][1]
print(type(s))

NameError: name 'EndDate' is not defined

In [13]:
from dateutil.relativedelta import relativedelta                                                                                                                  
from datetime import date 
d = variables_df['Coupon Start'][1]  
EndDate = variables_df['Expiration Date'][1]
quarter = relativedelta(months=3)                                                                                                                                 
while True: 
    print(d + quarter) 
    d += quarter 
    if d >= EndDate:
            break
d = d.strftime('%Y-%m-%d')
d = pd.DataFrame(cf,columns=['Cash Flow Dates Monthly'])
print(type(d))

TypeError: can only concatenate str (not "relativedelta") to str

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

variables_df['Start Date'] = pd.to_datetime(variables_df['Start Date'])
variables_df['Maturity'] = pd.to_datetime(variables_df['Maturity'])
variables_df['Coupon Start'] = pd.to_datetime(variables_df['Coupon Start'])
variables_df['Expiration Date'] = pd.to_datetime(variables_df['Expiration Date'])
variables_df['Cash Flow Period'] =  variables_df['Maturity'] - variables_df['Coupon Start'] 
print(variables_df['Start Date'][0])
print(type(variables_df['Start Date'][0]))
print(variables_df['Cash Flow Period'][0])
print(variables_df)

2023-02-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
731 days 00:00:00
   ID  Type    Name  Face Value  Coupon Rate  Coupon Period Currency  \
0   1  Bond  Bond A     10000.0         3.00             12      USD   
1   2  Bond  Bond B   1000000.0         3.25             12      EUR   
2   3  Bond  Bond C    100000.0         2.88             12      EUR   

  Start Date   Maturity Compound Coupon Start Expiration Date Cash Flow Period  
0 2023-02-01 2025-02-01        M   2023-02-01      2025-01-01         731 days  
1 2023-02-01 2028-02-01        M   2023-05-15      2028-01-01        1723 days  
2 2023-02-01 2033-02-01        M   2023-02-01      2033-01-01        3653 days  


In [15]:

# Issue: Need to find a way to convert variables_df['Cash Flow Period'][0].days into months
# without a float, meaning with the following modules
#from datetime import datetime
#from dateutil.relativedelta import relativedelta

def cf_calculator(ID, freq):
    for i in range(0,(variables_df['Cash Flow Period'][ID].days * freq)+1):
        if i < (variables_df['Cash Flow Period'][ID].days * freq):
            coupon = ((variables_df.iloc[ID,4]/100) * variables_df.iloc[ID,3]) / freq
            print(coupon)
        else: 
            coupon = (((variables_df.iloc[ID,4]/100) * variables_df.iloc[ID,3]) / freq) + variables_df.iloc[ID,3]
            print(coupon)
cf_calculator(0,12)


25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0


Let's focus on creating the proper DatimeTime timestamps which aline with the corresponding cash flow payments

In [16]:
from datetime import *; from dateutil.relativedelta import *
import calendar

def cshflwMonthly(ID,Months_Maturity):
    StartDate = variables_df['Coupon Start'][ID]
    Maturity = Months_Maturity
    EndDate = StartDate + relativedelta(months=Maturity)
    print(StartDate);print(EndDate)
    cf = pd.date_range(start=StartDate,end=EndDate,periods=Maturity,normalize=True)
    cf = cf.strftime('%Y-%m-%d')
    cf = pd.DataFrame(cf,columns=['Cash Flow Dates Monthly'])
    print(type(cf))
    
cshflwMonthly(0,12)


    

2023-02-01 00:00:00
2024-02-01 00:00:00
<class 'pandas.core.frame.DataFrame'>


In [17]:
from datetime import *; from dateutil.relativedelta import *
import calendar

def cshflwYearly(ID,Years_Maturity):
    StartDate = variables_df['Coupon Start'][ID]
    Maturity = Years_Maturity
    EndDate = StartDate + relativedelta(years=Maturity)
    print(StartDate);print(EndDate)
    cf = pd.date_range(start=StartDate,end=EndDate,periods=Maturity,normalize=True)
    cf = cf.strftime('%Y-%m-%d')
    cf = pd.DataFrame(cf,columns=['Cash Flow Dates Yearly'])
    print(cf)

    
    
cshflwYearly(0,2)

2023-02-01 00:00:00
2025-02-01 00:00:00
  Cash Flow Dates Yearly
0             2023-02-01
1             2025-02-01


In [18]:
from datetime import *; from dateutil.relativedelta import *
import calendar

def cshflwQuarterly(ID):
    StartDate = variables_df['Coupon Start'][ID]
    Maturity = 24
    Coupon = Maturity / 3
    EndDate = StartDate + relativedelta(months=Maturity)
    Quarter = relativedelta(months=3)
    print(StartDate);print(EndDate)
    cf = pd.date_range(start=StartDate,end=EndDate,freq='3M')
    cf = cf.strftime('%Y-%m-%d')
    cf = pd.DataFrame(cf,columns=['Cash Flow Dates Quarterly'])
    print(cf)
    
cshflwQuarterly(0)

2023-02-01 00:00:00
2025-02-01 00:00:00
  Cash Flow Dates Quarterly
0                2023-02-28
1                2023-05-31
2                2023-08-31
3                2023-11-30
4                2024-02-29
5                2024-05-31
6                2024-08-31
7                2024-11-30


In [19]:
StartDate = variables_df['Coupon Start'][0]
Maturity = 24
EndDate = StartDate + relativedelta(months=Maturity)
cf = pd.date_range(start=StartDate,end=EndDate)
cf = pd.DataFrame(cf,columns=['Cash Flow'])
print(cf)


     Cash Flow
0   2023-02-01
1   2023-02-02
2   2023-02-03
3   2023-02-04
4   2023-02-05
..         ...
727 2025-01-28
728 2025-01-29
729 2025-01-30
730 2025-01-31
731 2025-02-01

[732 rows x 1 columns]


In [20]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar


variables_df['Cash Flow Period'] =  variables_df['Maturity'] - variables_df['Coupon Start'] 

StartDate = variables_df['Coupon Start'][0].months
Maturity = 24
# Number of periods
Periods = Maturity/3
EndDate = StartDate + relativedelta(months=Maturity)

pd.date_range(start=StartDate,end=EndDate,periods=Periods)

print(StartDate)

for i in range (StartDate,EndDate):
    if i < EndDate:
        x = StartDate
        
    else: print("end") 

print(StartDate)

t00=variables_df['Coupon Start'][0]
t01=variables_df['Coupon Start'][0] + relativedelta(months=1)
t02=t01 + relativedelta(months=1)

print(t00,t01,t02)



AttributeError: 'Timestamp' object has no attribute 'months'

## Cash Flow Output Table
Table where cf_calculator() will output results and parameters will be used for identification
1. ID
2. Principal
3. Date
4. Cash Flow 'cf_calculator()'


In [ ]:
cshflwQuarterly(0)
cshflwYearly(0,2)    
cshflwMonthly(0,12)


In [ ]:
cf_calculator(0,12)